# BeKern Transposition Demo

This notebook demonstrates loading bekern notation from a .semantic file and using Verovio's transposition functionality.

---

## 1. Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional
import warnings
warnings.filterwarnings('ignore')

# Import existing demo utilities
from utils import render_music_score, bekern_to_kern

# Import Verovio
try:
    import verovio
    VEROVIO_AVAILABLE = True
except ImportError:
    VEROVIO_AVAILABLE = False

## 2. Helper Functions

In [ ]:
def load_bekern_file(file_path: str) -> str:
    """Load bekern notation from a .semantic file."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except:
        return ""

def render_with_transposition(kern_str: str, transpose_semitones: int = 0) -> Optional[np.ndarray]:
    """Render kern notation with optional transposition using Verovio."""
    if not VEROVIO_AVAILABLE:
        return None
    
    try:
        tk = verovio.toolkit()
        
        # Set options with transposition if specified
        options = {
            "pageWidth": 2100,
            "scale": 40,
            "adjustPageHeight": True,
            "footer": "none",
            "header": "none"
        }
        
        if transpose_semitones != 0:
            options["transpose"] = str(transpose_semitones)
        
        tk.setOptions(options)
        tk.loadData(kern_str)
        svg = tk.renderToSVG()
        
        # Use existing conversion from utils
        import cairosvg
        import cv2
        png_data = cairosvg.svg2png(bytestring=svg.encode('utf-8'), background_color='white')
        image_array = cv2.imdecode(np.frombuffer(png_data, np.uint8), -1)
        return cv2.cvtColor(image_array, cv2.COLOR_BGR2RGB)
        
    except:
        return None

def display_comparison(original_img, transposed_img, transpose_amount):
    """Display side-by-side comparison of original and transposed notation."""
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    if original_img is not None:
        axes[0].imshow(original_img)
        axes[0].set_title("Original Notation", fontsize=14)
    else:
        axes[0].text(0.5, 0.5, 'Rendering\nFailed', ha='center', va='center', 
                    transform=axes[0].transAxes)
    
    if transposed_img is not None:
        axes[1].imshow(transposed_img)
        direction = "Up" if transpose_amount > 0 else "Down"
        axes[1].set_title(f"Transposed {abs(transpose_amount)} Semitones {direction}", fontsize=14)
    else:
        axes[1].text(0.5, 0.5, 'Rendering\nFailed', ha='center', va='center', 
                    transform=axes[1].transAxes)
    
    for ax in axes:
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

## 3. Configuration

In [ ]:
# Configuration
SEMANTIC_FILE_PATH = "path/to/your/bekern_file.semantic"  # Update this path
TRANSPOSE_SEMITONES = 5  # Positive = up, negative = down

## 4. Load BeKern File

In [ ]:
# Load bekern file
bekern_content = load_bekern_file(SEMANTIC_FILE_PATH)

# Use example if file not found
if not bekern_content:
    bekern_content = """*clefF4 *clefG2 *k[b-] *k[b-] *M2/4 *M2/4 
8·F·L 16·r
8·c·J 8·A 16·ff·LL
8·C·L 16·ee
8·c·J 8·A 16·dd·JJ
= =
8·G·L 16·cc·LL
8·e·J 8·c 8·B·- 16·b·n
8·C·L 16·cc
8·e·J 8·c 8·B·- 16·a·JJ"""

## 5. Convert to Kern Format

In [ ]:
# Convert to kern format
kern_notation = bekern_to_kern(bekern_content)

## 6. Render Comparison

**Transposition**: Positive numbers transpose up, negative transpose down. Common intervals:
- 1=minor 2nd, 2=major 2nd, 3=minor 3rd, 5=perfect 4th, 7=perfect 5th, 12=octave

In [ ]:
# Render both versions
original_image = render_with_transposition(kern_notation, 0)
transposed_image = render_with_transposition(kern_notation, TRANSPOSE_SEMITONES)

# Display comparison
display_comparison(original_image, transposed_image, TRANSPOSE_SEMITONES)

## 7. Test Multiple Transpositions

In [ ]:
# Test multiple transpositions
test_values = [-7, -5, -3, -1, 0, 1, 3, 5, 7, 12]

for transpose_val in test_values:
    result = render_with_transposition(kern_notation, transpose_val)
    status = "✅" if result is not None else "❌"
    print(f"{transpose_val:+3d} semitones: {status}")

## 8. Manual Testing

In [ ]:
# Manual testing - change this value and re-run the cell
CUSTOM_TRANSPOSE = 7  # Change this to test different transpositions

# Render and display
custom_original = render_with_transposition(kern_notation, 0)
custom_transposed = render_with_transposition(kern_notation, CUSTOM_TRANSPOSE)
display_comparison(custom_original, custom_transposed, CUSTOM_TRANSPOSE)